# [TFIDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

## Vaex

In [9]:
import vaex
import sklearn.pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from vaex.ml.sklearn import Predictor
from goldilox import Pipeline


text = ['I love soup', 'you love soup']
y = ['me', 'you']
df = vaex.from_arrays(text=text, y=y)

sk_pipeline = sklearn.pipeline.Pipeline([('tfidf', TfidfVectorizer()), ('classifier', LogisticRegression())]).fit(df['text'].to_numpy(), df['y'].to_numpy())


@vaex.register_function()
def tfidf(ar):
    return sk_pipeline.predict(ar.tolist())

df.add_function('tfidf', tfidf)
df['prediction'] = df['text'].tfidf()

# Pipeline
pipeline = Pipeline.from_vaex(df)
assert pipeline.validate()
pipeline.inference({'text':'you and me'})

#,text,y,prediction
0,you and me,--,you


## Sklearn

In [13]:
import sklearn.pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from goldilox import Pipeline
import pandas as pd

text = ['I love soup', 'you love soup']
y = ['me', 'you']

df = pd.DataFrame({'text':text, "y":y})

sk_pipeline = sklearn.pipeline.Pipeline([('tfidf', TfidfVectorizer()), ('classifier', LogisticRegression())]).fit(df['text'], df['y'])

pipeline = Pipeline.from_sklearn(sk_pipeline, raw={'text':'example'})
assert pipeline.validate()
pipeline.inference({'text':'I like cats'})

,text,prediction
0,I like cats,me


# Serve

In [7]:
pipeline.save('pipeline.pkl')
print('Go to the fastapi docs here: http://127.0.0.1:5000/docs')
!gl serve 'pipeline.pkl'

Go to the fasgtapi docs here: http://127.0.0.1:5000/docs
[2021-11-26 12:10:04 +0100] [39494] [INFO] Starting gunicorn 20.1.0
[2021-11-26 12:10:04 +0100] [39494] [INFO] Listening at: http://127.0.0.1:5000 (39494)
[2021-11-26 12:10:04 +0100] [39494] [INFO] Using worker: uvicorn.workers.UvicornH11Worker
[2021-11-26 12:10:04 +0100] [39527] [INFO] Booting worker with pid: 39527
[2021-11-26 12:10:04 +0100] [39527] [INFO] Started server process [39527]
[2021-11-26 12:10:04 +0100] [39527] [INFO] Waiting for application startup.
[2021-11-26 12:10:04 +0100] [39527] [INFO] Application startup complete.
^C
[2021-11-26 12:10:21 +0100] [39494] [INFO] Handling signal: int
